# Download Streaming Data

First you have to download both your normal streaming data and your extended streaming data on Spotify:

    -> Go to your account information

    -> Go to Privacy Settings
    
    -> Request account data and extended streaming history

After you got the data, copy the files in the directory "Data/DataFromSpotify"
. With that all the paths, should work just fine.

# Preperations

Importing all the needed libraries

In [148]:
# Important all the needed libraries
import pandas as pd 
import json
import csv
import os
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

Setting up the Spotify API Client:

    -> Go to the Spotify Developer Site

    -> Create a new App

    -> Retrieve the needed Keys and insert them below

In [149]:
clientID = "Your Client ID"

secretID = "Cour Secret ID"

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=clientID, client_secret=secretID))

# Prepare the streaming Data

    -> Put all the streaming history files (the ones starting with Streaming_History_Audio) in the directory "Data/Streaming_History_Files" 
    
    -> The following code goes through all the files and merges them 

    -> It wil also create a new csv file called merged_streaming_history.csv which is stored in the data directory


In [150]:
# List the paths of your JSON files
streamingHistoryFilesDirectory = os.fsencode('Data/Streaming_History_Files')

# Define the path for the output CSV file
output_csv_file = 'Data/RawData/merged_streaming_history.csv' 

# List to store all data from JSON files
all_data = []

# Loop through each JSON file and extract data
for json_file in os.listdir(streamingHistoryFilesDirectory):
    file_path = os.path.join(streamingHistoryFilesDirectory, json_file)
    if os.path.isfile(file_path) and json_file.decode('utf-8').endswith('.json'):
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            all_data.extend(data)

# Check if there is any data to write to CSV
if not all_data:
    print("No data found in JSON files.")
else:
    # Determine the keys for CSV header (assuming each JSON object has the same structure)
    csv_header = all_data[0].keys()

    # Write data to CSV file
    with open(output_csv_file, 'w', newline='', encoding='utf-8') as f:
        csv_writer = csv.DictWriter(f, fieldnames=csv_header)

        # Write the CSV header
        csv_writer.writeheader()

        # Write each JSON object as a row in the CSV file
        csv_writer.writerows(all_data)

    print("JSON files merged and converted to CSV successfully!")

JSON files merged and converted to CSV successfully!


Create a pandas data frame for this data Set and clean the data:

    -> We want to have only tracks and no podcasts in this data set, we can also dropt the episode_name, episode_show_name and spotify_episode_uri column

    -> In this data set we have the Spotify URI, we want to add a column to have the song ID which is the second part of the Spotify URI (after the 'track:')

In [151]:
# Create the Dataframe
mergedData_df = pd.read_csv('Data/RawData/merged_streaming_history.csv')

# Drop the rows where the spotify_episode_uri has any values
mergedData_df = mergedData_df[mergedData_df['spotify_episode_uri'].isna()]

# Drop the columns we don't need anymore
mergedData_df.drop(['episode_name', 'episode_show_name', 'spotify_episode_uri'], axis=1, inplace=True)

# Create the songID column
mergedData_df['songID'] = mergedData_df['spotify_track_uri']
mergedData_df['songID'] = mergedData_df['songID'].str[-22:]

/var/folders/vz/zlsdllt500lgf0_qrzxkk3xw0000gn/T/ipykernel_14950/3368755055.py:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  mergedData_df = pd.read_csv('Data/RawData/merged_streaming_history.csv')


# Create the Master Data 

The Master Data is the Dataframe that we will add all the information to

In [152]:
# So far the master data is the merged data 
masterData_df = mergedData_df.copy()

# Library Data 

Append the information if the song is saved in my library.

    -> Load the library songs as a dataframe

    -> Loop through the streaming data and check if the song is in the library

    -> Add a column "in_library" with values True and False

In [153]:
with open('Data/DataFromSpotify/YourLibrary.json', 'r') as json_file:
    data = json.load(json_file)

library_df = pd.DataFrame(data['tracks'])


In [154]:
addedColumn = []

for trackName in masterData_df['master_metadata_track_name']:
    if trackName in set(library_df['track']): 
        addedColumn.append(True)
    else:
        addedColumn.append(False)

masterData_df['In Favourites'] = addedColumn


# Track Data

Now we perform some API requests to collect some more information about each song. We use the Spotify API to do that. 

We want to get the following information:

    -> Release Dat 
    -> Genre
    -> Popularity of the artist
    -> Popularity of song
    -> Is the song explicit?
    -> Id of song
    -> Name of song
    -> popularity of song

Please refer to this https://developer.spotify.com/documentation/web-api/reference/get-several-tracks for an explanation for every data point.

We can get information about 50 songs in one request. 

Therfore we make a list of lists, where each list contains 50 songs

In [155]:
# make a unique list of URIs
uniqueURIs = list(set(masterData_df['spotify_track_uri']))

# Aggregate those ids in a list of lists -> 50 ids in one list
listOfIDlists = []
listOfIDs = []
for index, uri in enumerate(uniqueURIs):
    listOfIDs.append(uri)
    if index % 49 == 0 or index == len(uniqueURIs) -1:
        if index != 0:
            listOfIDlists.append(listOfIDs)
            listOfIDs = []

Now we loop through listOfIDlists and make an API call for each. 

We store the reponses in a dedicated json file. Path: /Data/RawData/songInformation.json

In [156]:
responses = []
badReponses = []
for index, uriList in enumerate(listOfIDlists):
    # ToDo Delete this line!
    if index < 2:
        try:
            response = sp.tracks(uriList)
            time.sleep(10)
            responses.append(response)
        except Exception as e:
            badReponses.append(uriList)
            print(e)

# Create a json file to store the responses
with open('Data/RawData/songInformation.json', 'w') as json_file:
    json.dump(responses, json_file, indent=4)   

Now we have the information, but we need to extract the relevant information.

In [157]:
# Now we have the reponses and want to extract the data an convert it to a dataframe
listOfRows = []
counter = 0
for trackList in responses:
    for track in trackList['tracks']:
        songID = track['id']
        songName = track['name']
        try:
            songDuration = track['duration_ms']
        except KeyError:
            songDuration = 1
        try:
            songPopularity = track['popularity']
        except KeyError:
            songPopularity = 101
        try:
            explicit = track['explicit']
        except KeyError:
            explicit = False
        try:
            releaseDate = track['album']['release_date'][0:4]
        except KeyError:
            releaseDate = '9999'
        try:
            if len(track['artists']) > 0:
                artist_name = track['artists'][0]['name']
                artist_uri = track['artists'][0]['uri']
            else:
                artist_name = 'Artist Not Found'
                artist_uri = 'Artist Not Found'
        except KeyError:
            artist = 'Artist Not Found'
            artist_uri = 'Artist Not Found'
        dictToAdd = {
            'songID': songID,
            'name': songName,
            'duration_ms': songDuration,
            'song_popularity': songPopularity,
            'song_explicit': explicit,
            'release_Date': releaseDate,
            'first_artist': artist_name,
            'artist_uri': artist_uri,
            #'genreOfArtist': genreOfArtis
        }
        listOfRows.append(dictToAdd)
        counter = counter + 1

df_songData = pd.DataFrame(listOfRows)

# Store the extracted data frame as a single csv file to store it permanenetly
df_songData.to_csv('Data/RawData/songInformation_extracted.csv', index=False)

# Track Analysis

In the following we will get data of an audio analysis of each track. We do this by making another Spotify API request. We will get the following information for each song:
    -> danceability
    -> energy
    -> key
    -> loudness
    -> mode
    -> speechiness
    -> acousticness
    -> instrumentalness
    -> liveness
    -> valence
    -> tempo
    -> type
    -> id
    -> uri
    -> track_href
    -> analysis_url
    -> duration_ms
    -> time_signature

Please refer to https://developer.spotify.com/documentation/web-api/reference/get-several-audio-features for an explanation of every data point.

We can get the audio analysis for 100 songs in one request. So we take the same approach as before, but now we make a list of lists, where each inner list contains 100 songs. 

In [158]:
# Aggregate those ids in a list of lists -> 100 ids in one list
listOfIDlists_audioAnalysis = []
listOfIDs_audioAnalysis = []
for index, uri in enumerate(uniqueURIs):
    listOfIDs_audioAnalysis.append(uri)
    if index % 99 == 0 or index == len(uniqueURIs) -1:
        if index != 0:
            listOfIDlists_audioAnalysis.append(listOfIDs_audioAnalysis)
            listOfIDs_audioAnalysis = []

Now we loop through listOfIDlists_audioAnalysis and make an API call for each. 

We store the reponses in a dedicated json file. Path: /Data/RawData/song_audioAnalysis.json

In [159]:
responses_audioAnalysis = []
badReponses_audioAnalysis = []
for index, uriList in enumerate(listOfIDlists_audioAnalysis):
    # ToDo Delete this line!
    if index < 2:
        try:
            response_audioAnalysis = sp.audio_features(tracks = uriList)
            time.sleep(10)
            responses_audioAnalysis.append(response_audioAnalysis)
        except Exception as e:
            badReponses_audioAnalysis.append(uriList)
            print(e)

with open('Data/RawData/song_audioAnalysis.json', 'w') as json_file:
    json.dump(responses_audioAnalysis, json_file, indent=4)   

In [160]:
listOfDataFrames_audioAnalysis = []
for response_forList in responses_audioAnalysis:
    listOfDataFrames_audioAnalysis.append(pd.DataFrame(response_forList))

df_songAnalysis = pd.concat(listOfDataFrames_audioAnalysis, ignore_index=True)

# Putting it all together

Now we have all the information and now we just have to join the dataframes. With data sets from the API calls we will do a left join on the songID where the masterData is the left part

### Merge 1

We will append the information for each track to the master data. 

In [161]:

commonKey = 'songID'
masterData_df = pd.merge(masterData_df, df_songData, on=commonKey, how='left')

### Merge 2

In this merge we will append the informaiton of the audio analysis of each song to the master data.


In [162]:
# First rename the id column of the audio analysis to songID
df_songAnalysis['songID'] = df_songAnalysis['id']
df_songAnalysis.drop(['id'], axis=1, inplace=True)


In [163]:
commonKey = 'songID'
masterData_df = pd.merge(masterData_df, df_songAnalysis, on=commonKey, how='left')

# Finished

Now we export the master data as a csv file and we can analyse and visualize it in Looker Studio.

In [164]:
masterData_df.to_csv('MasterData.csv', index=False)
masterData_df

,ts,username,platform,ms_played,conn_country,ip_addr_decrypted,user_agent_decrypted,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms_y,time_signature
0,2016-07-05T14:31:36Z,julhallo0,"iOS 9.3.2 (iPad5,3)",336453,DE,92.211.47.167,unknown,In The Air Tonight - 2015 Remastered,Phil Collins,Face Value (Deluxe Editon),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-07-05T21:16:00Z,julhallo0,"iOS 9.3.2 (iPad5,3)",134243,DE,92.211.47.167,unknown,Miss Murray - Radio Mix,Timid Tiger,Timid Tiger & A Pile Of Pipers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-07-05T21:19:02Z,julhallo0,"iOS 9.3.2 (iPad5,3)",180333,DE,92.211.47.167,unknown,Random,G-Eazy,When It's Dark Out,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-07-05T21:23:54Z,julhallo0,"iOS 9.3.2 (iPad5,3)",247866,DE,92.211.47.167,unknown,Panda,Desiigner,Panda,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-07-05T21:26:48Z,julhallo0,"iOS 9.3.2 (iPad5,3)",173986,DE,92.211.47.167,unknown,One Dance,Drake,Views,...,0.00423,0.351,0.371,103.989,audio_features,spotify:track:1xznGGDReH1oQq0xzbwXa3,https://api.spotify.com/v1/tracks/1xznGGDReH1o...,https://api.spotify.com/v1/audio-analysis/1xzn...,173987.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33589,2016-07-05T14:09:35Z,julhallo0,"iOS 9.3.2 (iPad5,3)",236001,DE,92.211.47.167,unknown,CAN'T STOP THE FEELING! (from DreamWorks Anima...,Justin Timberlake,CAN'T STOP THE FEELING! (from DreamWorks Anima...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33590,2016-07-05T14:13:17Z,julhallo0,"iOS 9.3.2 (iPad5,3)",222160,DE,92.211.47.167,unknown,This Is What You Came For,Calvin Harris,This Is What You Came For,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33591,2016-07-05T14:16:12Z,julhallo0,"iOS 9.3.2 (iPad5,3)",173986,DE,92.211.47.167,unknown,One Dance,Drake,Views,...,0.00423,0.351,0.371,103.989,audio_features,spotify:track:1xznGGDReH1oQq0xzbwXa3,https://api.spotify.com/v1/tracks/1xznGGDReH1o...,https://api.spotify.com/v1/audio-analysis/1xzn...,173987.0,4.0
33592,2016-07-05T14:21:08Z,julhallo0,"iOS 9.3.2 (iPad5,3)",295333,DE,92.211.47.167,unknown,Who Wants To Live Forever - Remastered 2011,Queen,Greatest Hits II,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
